<a href="https://colab.research.google.com/github/SiyagJatin/Dog_cat_classification/blob/main/dog_cat_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [34]:
#!/bin/bash
!kaggle datasets download shaunthesheep/microsoft-catsvsdogs-dataset

Dataset URL: https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset
License(s): other
microsoft-catsvsdogs-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [35]:
import zipfile
zip_ref =zipfile.ZipFile('/content/microsoft-catsvsdogs-dataset.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [36]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [58]:
data_dir='/content/PetImages'

In [59]:
image_exts=['jpg','jpeg','png','bmp']
for image_class in os.listdir(data_dir):
  for image in os.listdir(os.path.join(data_dir,image_class)):
    image_path=os.path.join(data_dir,image_class,image)
    try:
      img=cv2.imread(image_path)
      with Image.open(image_path) as img:
        tip = img.format.lower()
      if tip not in image_exts:
        print('image not in ext list {}'.format(image_path))
        os.remove(image_path)
    except Exception as e:
      tip = None
      print('Issue with image {}'.format(image_path))
      #os.remove(image_path)

Issue with image /content/PetImages/Cat/666.jpg
Issue with image /content/PetImages/Cat/Thumbs.db
Issue with image /content/PetImages/Dog/Thumbs.db
Issue with image /content/PetImages/Dog/11702.jpg


In [60]:
import os
from PIL import Image # Install with: pip install Pillow
import logging

# Set the path to your main dataset directory
data_dir = './your_dataset_directory/'
problematic_files = []

# Configure logging to suppress less severe messages from Pillow
logging.basicConfig(level=logging.ERROR)

print(f"Scanning directory: {data_dir}")

for root, _, files in os.walk(data_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            try:
                image_path = os.path.join(root, file)
                with Image.open(image_path) as img:
                    # 'L' is grayscale (1 channel), 'RGB' is 3 channels, 'RGBA' is 4
                    # Anything else, like 'LA' (grayscale + alpha, 2 channels), is a problem.
                    if img.mode not in ['L', 'RGB', 'RGBA']:
                        problematic_files.append((image_path, img.mode))
            except Exception as e:
                print(f"Could not read {image_path}: {e}")

if problematic_files:
    print("\nFound problematic files:")
    for path, mode in problematic_files:
        print(f"- Path: {path}, Mode: {mode}")
else:
    print("\nNo problematic files found.")

Scanning directory: ./your_dataset_directory/

No problematic files found.


In [ ]:
import tensorflow as tf
import os

data_dir = '/content/PetImages'
for image_class in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, image_class)
    if os.path.isdir(class_dir):
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            try:
                # Read the image file
                img_bytes = tf.io.read_file(image_path)
                # Try to decode the image
                tf.io.decode_image(img_bytes)
            except tf.errors.InvalidArgumentError as e:
                print(f"Removing problematic image: {image_path} due to error: {e}")
                os.remove(image_path)

In [61]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.applications.vgg16 import VGG16

In [62]:
conv_base = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(256,256,3)
)

In [63]:
conv_base.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 256, 256, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 128, 128, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 32, 32, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [64]:
model=Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [67]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 8, 8, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │     8,388,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,103,809 (88.13 MB)

 Trainable params: 8,389,121 (32.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [66]:
conv_base.trainable = False

In [73]:
#Generators
train_ds = tf.keras.utils.image_dataset_from_directory(
    '/content/PetImages',
    validation_split = 0.2,
    subset = 'training',
    labels = 'inferred',
    seed=123,
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256),
    color_mode = 'rgb'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    '/content/PetImages',
    validation_split = 0.2,
    subset = 'validation',
    labels = 'inferred',
    seed=123,
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256),
    color_mode = 'rgb'
)

Found 24940 files belonging to 2 classes.
Using 19952 files for training.
Found 24940 files belonging to 2 classes.
Using 4988 files for validation.


In [74]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [75]:
history = model.fit(train_ds,epochs=10,validation_data=val_ds)

Epoch 1/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 194s 307ms/step - accuracy: 0.9571 - loss: 0.6493 - val_accuracy: 0.9715 - val_loss: 0.1109
Epoch 2/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 185s 282ms/step - accuracy: 0.9866 - loss: 0.0553 - val_accuracy: 0.9783 - val_loss: 0.0993
Epoch 3/10


KeyboardInterrupt: 